In [69]:
!pip install requests vaderSentiment python-dotenv sqlalchemy finnhub-python psycopg2-binary



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [70]:
import os
import requests
from datetime import datetime
from sqlalchemy import create_engine, text
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from dotenv import load_dotenv
import finnhub

In [71]:
load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL")
FINNHUB_API_KEY = os.getenv("FINNHUB_API_KEY")
GNEWS_API_KEY = os.getenv("GNEWS_API_KEY")

assert DATABASE_URL and FINNHUB_API_KEY and GNEWS_API_KEY, "Missing environment variables!"

engine = create_engine(DATABASE_URL)
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)
vader = SentimentIntensityAnalyzer()



In [72]:
def analyze_sentiment(text):
    score = vader.polarity_scores(text)['compound']
    label = 'POS' if score > 0.05 else 'NEG' if score < -0.05 else 'NEU'
    return score, label, 'VADER', None

def insert_sentiment(data_dict):
    with engine.begin() as conn:
        query = text("""
            INSERT INTO sentiment_data (
                ticker, source, api_vendor, date, headline, sentiment_score,
                sentiment_label, article_url, author, platform_tags,
                model_used, confidence
            ) VALUES (
                :ticker, :source, :api_vendor, :date, :headline, :sentiment_score,
                :sentiment_label, :article_url, :author, :platform_tags,
                :model_used, :confidence
            )
        """)
        conn.execute(query, data_dict)


In [73]:
def fetch_finnhub_sentiment(ticker):
    try:
        news = finnhub_client.company_news(ticker, _from='2024-06-01', to=datetime.today().strftime('%Y-%m-%d'))
        for item in news:
            if not item.get('headline'):
                continue

            score, label, model, conf = analyze_sentiment(item['headline'])
            data = {
                'ticker': ticker,
                'source': item.get('source', 'Finnhub'),
                'api_vendor': 'Finnhub',
                'date': datetime.fromtimestamp(item['datetime']).date(),
                'headline': item['headline'],
                'sentiment_score': score,
                'sentiment_label': label,
                'article_url': item.get('url'),
                'author': item.get('source'),
                'platform_tags': 'Finnhub',
                'model_used': model,
                'confidence': conf,
                'created_at': datetime.utcnow().isoformat()
            }
            insert_sentiment(data)
        print(f"Inserted sentiment records for {ticker} from Finnhub")
    except Exception as e:
        print(f"Error fetching Finnhub data for {ticker}: {e}")


In [75]:
def fetch_gnews_sentiment(ticker):
    url = f"https://gnews.io/api/v4/search?q={ticker}&token={GNEWS_API_KEY}&lang=en&max=10"
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception(f"GNews API failed: {response.status_code} - {response.text}")
    
    articles = response.json().get("articles", [])
    
    for article in articles:
        score, label, model, conf = analyze_sentiment(article["title"])
        data = {
            "ticker": ticker,
            "source": article.get("source", {}).get("name", "GNews"),
            "api_vendor": "GNews API",
            "date": datetime.strptime(article["publishedAt"][:10], "%Y-%m-%d").date(),
            "headline": article["title"],
            "sentiment_score": score,
            "sentiment_label": label,
            "article_url": article["url"],
            "author": article.get("author") or "Unknown",
            "platform_tags": "GNews",
            "model_used": model,
            "confidence": conf
        }
        insert_sentiment(data)
    
    print(f"✅ GNews: Inserted sentiment records for {ticker}")


tickers = ['NVDA']

for ticker in tickers:
    fetch_finnhub_sentiment(ticker)
    fetch_gnews_sentiment(ticker)



Inserted sentiment records for NVDA from Finnhub
✅ GNews: Inserted sentiment records for NVDA
